## import libraries

In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## get data from Internet

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())

## Processing 1- getting raw table

In [3]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
             
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

## Processing 2 : joining postal codes wit more than 1 neighbour

In [4]:

unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


## create a correct Dataframe

In [5]:
toronto_dict = {'Borough':Borough_u,'Neighbourhood':Neighbourhood_u,'Postcode':Postcode_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Borough,Neighbourhood,Postcode
0,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P
1,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que...",M8Y
2,East York,Leaside,M4G
3,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K
4,Scarborough,"Cliffcrest, Cliffside",M1M
5,East York,East Toronto,M4J
6,Scarborough,"Rouge, Malvern",M1B
7,North York,"Emery, Humberlea",M9M
8,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V
9,West Toronto,"Parkdale, Roncesvalles",M6R


In [6]:
df_toronto.shape

(77, 3)

**NB: the number of unique postal codes is 77

## export dataset as csv

In [7]:
df_toronto.to_csv(r'C:\Users\w5080797\Documents\Personal\IBM_qualification\df_toronto.csv')